# Parameters

In [1]:
# Variables
SAMPLE_SIZE_EVAL=500_000 # reduce size of evaluation data
SAMPLE_SIZE_TRAIN=1_000_000 # reduce size of training data

# drop out all events with Energies outside the range
LOWER_LIMIT = 100
upper_limit = 10**5

# Variables: NN
ONE_MODEL = True
NUM_BINS = 10 # number of bins (energy classes), output_shape
GPU_ID = 1

# Imports and setup for Numpy RNG

In [2]:
# workaround if you have not installed CherenkovDeconvolution.py
import sys

sys.path.insert(0,'/net/nfshome/home/shaefs/CherenkovDeconvolution.py')

In [3]:
import numpy as np
np.random.seed(42)

In [4]:
# data management
import os

# data processing
import pandas as pd
import tensorflow as tf
from sklearn import utils

# Visualisation tools
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# DSEA
import cherenkovdeconvolution as cd
import cherenkovdeconvolution.stepsize as stepsize

# Read in data

In [5]:
# Feature Selection
feature_list = ['SplineMPEDirectHitsICE.n_dir_doms', 
                        'VariousVariables.Cone_Angle',
                        'SplineMPECramerRaoParams.variance_theta',
                        'Borderness.Q_ratio_in_border',
                        'SplineMPETruncatedEnergy_SPICEMie_BINS_MuEres.value',
                        'SplineMPETruncatedEnergy_SPICEMie_DOMS_Neutrino.energy',
                        'SplineMPEDirectHitsICB.n_late_doms',
                        'Dustyness.n_doms_in_dust',
                        'LineFitGeoSplit1Params.n_hits',
                        'SplineMPEDirectHitsICC.dir_track_hit_distribution_smoothness',
                        'SPEFit2GeoSplit1BayesianFitParams.logl',
                        'SplineMPECharacteristicsIC.avg_dom_dist_q_tot_dom']
#feature_list = ['MCPrimary.energy']



In [6]:
# just read-in necessary columns
fields = feature_list + ['MCPrimary.energy']

In [7]:
# read in data
file_path = '/net/big-tank/POOL/users/lkardum/new_mc_binning.csv'
df_raw = pd.read_csv(file_path, skipinitialspace=True, usecols=fields)
df_raw

,SplineMPECramerRaoParams.variance_theta,SplineMPETruncatedEnergy_SPICEMie_DOMS_Neutrino.energy,MCPrimary.energy,SplineMPEDirectHitsICC.dir_track_hit_distribution_smoothness,SplineMPETruncatedEnergy_SPICEMie_BINS_MuEres.value,SplineMPECharacteristicsIC.avg_dom_dist_q_tot_dom,LineFitGeoSplit1Params.n_hits,SplineMPEDirectHitsICB.n_late_doms,SplineMPEDirectHitsICE.n_dir_doms,Borderness.Q_ratio_in_border,SPEFit2GeoSplit1BayesianFitParams.logl,Dustyness.n_doms_in_dust,VariousVariables.Cone_Angle
0,0.000138,20601.845970,3086.645078,-0.147665,0.397322,41.003142,19.0,34.0,20.0,0.101743,158.569898,0.175000,0.022686
1,0.003101,9461.487291,193.041166,0.344174,0.545559,50.324590,39.0,13.0,11.0,0.011322,272.450606,0.057143,0.097690
2,0.000325,43222.679757,862.503704,0.281681,0.463602,39.265226,21.0,20.0,17.0,0.618235,73.385557,0.217391,0.049914
3,0.000066,9929.465219,288.757353,0.222210,0.397322,16.576652,12.0,16.0,13.0,0.041282,81.277756,0.875000,0.023061
4,0.001243,14979.867844,1838.155763,-0.233723,0.545559,11.832243,16.0,16.0,13.0,0.979636,90.623889,0.294118,0.222784
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13336408,0.002613,28988.905556,2680.561232,-0.204343,0.545559,24.473264,5.0,13.0,11.0,0.000000,56.708233,0.000000,0.149460
13336409,0.000112,13010.699980,1695.794619,0.567237,0.342283,41.072783,110.0,58.0,23.0,0.002037,302.582772,0.555556,0.134817
13336410,0.010503,15232.377792,579.684909,0.317987,0.545559,45.828011,13.0,19.0,17.0,0.274274,108.338541,0.125000,0.090354
13336411,0.004765,19444.610677,205.607328,0.341417,0.545559,42.645669,13.0,12.0,8.0,0.000000,107.530913,0.000000,0.128185


In [8]:
# drop critical rows
df = df_raw.drop([ 1007671,  1426233,  2304853,  2715790,  3674355,  3741687, 4178063,  4969266,  5038333,  5334552,  5589516,  5863719,
 5978972,  7006367,  7281704,  7509650,  8380383,  8758113,9043798, 10280382, 11179530, 11184928, 11332586, 11797767,12253944], axis = 0)

# substitute NaNs with extreme Value
df.fillna(value =-100000, inplace = True)

# Discretize the target Neutrino Energy

In [9]:
# throw out extreme high and low energy neutrinos
df = df[(df['MCPrimary.energy']<upper_limit) & (df['MCPrimary.energy']>LOWER_LIMIT)]
df

,SplineMPECramerRaoParams.variance_theta,SplineMPETruncatedEnergy_SPICEMie_DOMS_Neutrino.energy,MCPrimary.energy,SplineMPEDirectHitsICC.dir_track_hit_distribution_smoothness,SplineMPETruncatedEnergy_SPICEMie_BINS_MuEres.value,SplineMPECharacteristicsIC.avg_dom_dist_q_tot_dom,LineFitGeoSplit1Params.n_hits,SplineMPEDirectHitsICB.n_late_doms,SplineMPEDirectHitsICE.n_dir_doms,Borderness.Q_ratio_in_border,SPEFit2GeoSplit1BayesianFitParams.logl,Dustyness.n_doms_in_dust,VariousVariables.Cone_Angle
0,0.000138,20601.845970,3086.645078,-0.147665,0.397322,41.003142,19.0,34.0,20.0,0.101743,158.569898,0.175000,0.022686
1,0.003101,9461.487291,193.041166,0.344174,0.545559,50.324590,39.0,13.0,11.0,0.011322,272.450606,0.057143,0.097690
2,0.000325,43222.679757,862.503704,0.281681,0.463602,39.265226,21.0,20.0,17.0,0.618235,73.385557,0.217391,0.049914
3,0.000066,9929.465219,288.757353,0.222210,0.397322,16.576652,12.0,16.0,13.0,0.041282,81.277756,0.875000,0.023061
4,0.001243,14979.867844,1838.155763,-0.233723,0.545559,11.832243,16.0,16.0,13.0,0.979636,90.623889,0.294118,0.222784
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13336408,0.002613,28988.905556,2680.561232,-0.204343,0.545559,24.473264,5.0,13.0,11.0,0.000000,56.708233,0.000000,0.149460
13336409,0.000112,13010.699980,1695.794619,0.567237,0.342283,41.072783,110.0,58.0,23.0,0.002037,302.582772,0.555556,0.134817
13336410,0.010503,15232.377792,579.684909,0.317987,0.545559,45.828011,13.0,19.0,17.0,0.274274,108.338541,0.125000,0.090354
13336411,0.004765,19444.610677,205.607328,0.341417,0.545559,42.645669,13.0,12.0,8.0,0.000000,107.530913,0.000000,0.128185


In [10]:
## log-scaled Binning
bins = np.logspace(np.log10(LOWER_LIMIT), np.log10(upper_limit), NUM_BINS+1)
bins

array([   100.        ,    199.5262315 ,    398.10717055,    794.32823472,
         1584.89319246,   3162.27766017,   6309.5734448 ,  12589.25411794,
        25118.8643151 ,  50118.72336273, 100000.        ])

In [11]:
# new column with discretized energies

df['E_discr'] = pd.cut(df['MCPrimary.energy'], bins=bins, labels=range(len(bins)-1))
df['E_discr'].value_counts()

1    3265825
2    2971469
3    2110752
0    1939697
4    1327291
5     776316
6     435447
7     239490
8     128030
9      67777
Name: E_discr, dtype: int64

# Create target dataset

In [12]:
X = df[feature_list].to_numpy()
X.shape

(13262094, 12)

In [13]:
y = df['E_discr'].to_numpy()
y.shape

(13262094,)

In [14]:
# subsample size

def create_dataset(X, y, subset=0, validation_split=0.3, rng_seed=42):
    # X,y: numpy Array
    # subset: size of the subset (Int)
    # validation_split: (0, 1)
    rng = np.random.default_rng(rng_seed)
    perm = rng.permutation(len(X))

    if subset:
        perm = perm[0:subset]

    limit = int(len(perm)*(1-validation_split))

    X_train = X[perm[:limit]]
    X_eval = X[perm[limit:]]
    y_train = y[perm[:limit]]
    y_eval = y[perm[limit:]]

    return X_train, y_train, X_eval, y_eval

In [15]:
X_train, y_train, X_eval, y_eval = create_dataset(X, y, validation_split=0.1)
print(X_train.shape, y_train.shape, X_eval.shape, y_eval.shape)

(11935884, 12) (11935884,) (1326210, 12) (1326210,)


In [16]:
# reduce size of train set
y_train = y_train[0:SAMPLE_SIZE_TRAIN]
X_train = X_train[0:SAMPLE_SIZE_TRAIN]
X_train.shape, y_train.shape

((1000000, 12), (1000000,))

In [17]:
# reduce size of test set
y_eval = y_eval[0:SAMPLE_SIZE_EVAL]
X_eval = X_eval[0:SAMPLE_SIZE_EVAL]
X_eval.shape, y_eval.shape

((500000, 12), (500000,))

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)

# Build Neural Network

In [19]:
from tensorflow.keras import backend as K

In [20]:
# define metric: chi-square distance
def chi2(y_true, y_pred):
    f_true = K.sum(y_true, axis=0)/2048
    f_pred = K.sum(y_pred, axis=0)/2048

    num = (f_true - f_pred)*(f_true - f_pred)
    den = f_true + f_pred
    return 0.5 * K.sum(num/den)

In [21]:
# define metric: chi-square distance
def chi2(y_true, y_pred):
    f_true = tf.reduce_sum(y_true, 0)/2048
    f_pred = tf.reduce_sum(y_pred, 0)/2048

    num = tf.square(f_true - f_pred)
    den = f_true + f_pred
    return 0.5 * tf.reduce_sum(num/den, axis=-1)

In [22]:
# neural network
def make_model(num_features, num_classes, learning_rate=0.0005):

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(120, input_shape=num_features, activation='relu'))
    #model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(240, activation='relu'))
    #model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(120, activation='relu'))
    #model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(12, activation='relu'))
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

    # compile the NN
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy()
    acc = tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None)

    model.compile(optimizer=opt, loss=loss, metrics=[acc, chi2])

    return model

# Define Classifier (Wrapper for DSEA/Sklearn)

In [23]:
class MyClassifier():
    def __init__(self, batch_size=2048, epochs=5, learning_rate=0.0005, one_model=True):
        """
        one_model: If True dsea will train only one model instead of generating a new one in each dsea iteration
        batchsize, epoch, learning_rate: model parameters
        """
        self.Niter = 0
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.model = None
        self.one_model = one_model
        self.history = [] # save model_history for each dsea iteration

    def fit(self, X, y, sample_weight=None):
        # print iteration num
        self.Niter += 1
        print(f'\nNumber iteration in DSEA: {self.Niter}')

        """
        y is NOT one-hot encoded yet
        """
        y_hot = np.zeros((y.size, y.max()+1))
        y_hot[np.arange(y.size),y] = 1
        
        # create new model if (one_model=False) or (one_model = True and there exist no model yet)
        if self.model is None or self.one_model is False:
            self.model = make_model(num_features=(len(feature_list), ), num_classes=y.max()+1, learning_rate=self.learning_rate)
        history = self.model.fit(X, y_hot, sample_weight=sample_weight, batch_size=self.batch_size, epochs=self.epochs)
        self.history.append(history)

        return self
    
    def predict_proba(self, X):
        return self.model.predict(X)

    def get_model(self):
        return self.model

    def get_model_history(self):
        list_loss = []
        list_acc = []
        list_chi = []

        for history in self.history:
            list_loss += history.history['loss']
            list_acc += history.history['categorical_accuracy']
            list_chi += history.history['chi2']

        list_loss = np.array(list_loss, dtype='float32')
        list_acc = np.array(list_acc, dtype='float32')
        list_chi = np.array(list_chi, dtype='float32')


        return list_loss, list_acc, list_chi

# True distribution

In [24]:
_, f_true = np.unique(y_eval, return_counts=True)
f_true = f_true/len(y_eval)
f_true

array([0.146518, 0.24642 , 0.223364, 0.159558, 0.099718, 0.05844 ,
       0.032956, 0.018036, 0.009848, 0.005142])

# Grid Search: Parameter space

In [25]:
epoch_space = [1, 5, 10, 25, 35, 50, 60, 75, 85, 100]
iter_space = [1, 3, 6, 8, 10, 12, 16, 20]
param_space = [(ep,it) for ep in epoch_space for it in iter_space]
len(param_space)

80

# Grid Search: Loop


In [26]:
ONE_MODEL, GPU_ID

(True, 1)

In [27]:
with tf.device(f'/gpu:{GPU_ID}'):
    for Nepoch, Niter in param_space:
        print(f'\n----Num epochs: {Nepoch}, Num iterations: {Niter}----')
        
        # if model exist: skip
        if os.path.isfile(f'gridsearch_results_subsample/one_model_{ONE_MODEL}/models/{Niter}it_{Nepoch}ep.h5'): # if model exist: skip
            print('Parameters already checked!')
            continue

        # create classifier and train in dsea
        cf = MyClassifier(batch_size=2048, epochs=Nepoch, learning_rate=0.0005, one_model=ONE_MODEL)
        f_dsea = cd.dsea(X_eval, X_train, y_train, cf, K=Niter)

        # get history
        loss_train, acc_train, chi2_train = cf.get_model_history()
        # append results to a csv file
        data = {'Epochs':[Nepoch], 'Iterations':[Niter], 'f_dsea':[f_dsea], 'f_true':[f_true], 'loss_train':[loss_train], 'acc_train':[acc_train], 'chi2_train':[chi2_train]}
        df_res = pd.DataFrame(data=data)
        df_res.to_csv(f'gridsearch_results_subsample/one_model_{ONE_MODEL}/results.csv', mode='a', index=False, header=False)

        # save the model weights
        cf.get_model().save_weights(f'gridsearch_results_subsample/one_model_{ONE_MODEL}/models/{Niter}it_{Nepoch}ep.h5')    


----Num epochs: 1, Num iterations: 1----
Parameters already checked!

----Num epochs: 1, Num iterations: 3----
Parameters already checked!

----Num epochs: 1, Num iterations: 6----
Parameters already checked!

----Num epochs: 1, Num iterations: 8----
Parameters already checked!

----Num epochs: 1, Num iterations: 10----
Parameters already checked!

----Num epochs: 1, Num iterations: 12----
Parameters already checked!

----Num epochs: 1, Num iterations: 16----
Parameters already checked!

----Num epochs: 1, Num iterations: 20----
Parameters already checked!

----Num epochs: 5, Num iterations: 1----
Parameters already checked!

----Num epochs: 5, Num iterations: 3----
Parameters already checked!

----Num epochs: 5, Num iterations: 6----
Parameters already checked!

----Num epochs: 5, Num iterations: 8----
Parameters already checked!

----Num epochs: 5, Num iterations: 10----
Parameters already checked!

----Num epochs: 5, Num iterations: 12----
Parameters already checked!

----Num epoch

In [28]:
# release gpu memory
from numba import cuda 
device = cuda.get_current_device()
device.reset()